# Pascal VOC 2006 Dataset Parsing

## Before start

<br>

### 우리는 지금 2class data 에 대해서만 학습시킬 것임. (car, people)

<br>

물론 실제 상황에서 2개의 class 에 대한 학습만으로는 터무니없이 부족함. <br>
하지만 하드웨어 및 파이의 성능을 고려할 때 큰 모델은 올리기가 어려움. <br>
따라서 ImageSets 의 폴더의 제목 중, bicycle, bus, car, mortorbike, person 을 제외한 모든 다른 폴더로 이동.

In [27]:
import os
import shutil

train_or_test = "test"
ImageSets_path_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/ImageSets"
PNGImages_path_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/PNGImages"
Annotations_path_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/Annotations"

Img_newpath_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/newPNGImages"
Annotation_temp_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/tempAnnotations"
Annotation_newpath_here = "C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/"+train_or_test+"/newAnnotations"

matching_dic = {'car' : 'car', 'bus' : 'car', 'person' : 'person'} # string from : class to

In [28]:
def read_filedata(full_fname):
    file=open(full_fname,'r')
    L = []
    while (1):
        line=file.readline()
        try:escape=line.index('\n')
        except:escape=len(line)    
        if line:
            L.append(line[0:escape])
        else:
            break
    file.close()
    return L

In [29]:
def move_file_to_anotherfolder(full_fname, full_folderpath, fname):
    if os.path.isfile(full_fname) == True :
        os.rename(full_fname, full_folderpath + '/' + fname)
    else :
        pass

In [30]:
def isStringExist(matching_dic, string) :    
    for key in matching_dic :
        if key in string :
            return matching_dic[key]
    else :
        return None

In [31]:
def bboxLocationParser(front_back_back_string) :
    
    data = front_back_back_string.split('-')
    topleft = tuple(map(int, data[0].replace(' ','').replace('(','').replace(')','').split(',')))
    bottomright = tuple(map(int, data[1].replace(' ','').replace('(','').replace(')','').split(',')))
#    print(topleft, bottomright)
    
    dic = {
        'xmin': topleft[0], # x left
        'ymin': topleft[1], # y top
        'xmax': bottomright[0], # x right
        'ymax': bottomright[1], # y bottom
    }
    
    return dic, topleft, bottomright

In [32]:
objecttype_name_list = []
for root, dirs, files in os.walk(ImageSets_path_here):
    print(root)
    #--------------------------------#
    #                                #
    #--------------------------------#
    
    for fname in files:
        full_fname = root + '/' + fname
        print(full_fname)
        li = read_filedata(full_fname)
        for data in li:
            name_val = data.split(' ')
            if len(name_val) == 3 :
                del name_val[1]
            if len(name_val) <= 1 :
                pass
            else :
                if name_val[1] == '1' :
                    # print('image name : ',name_val[0], ', object exist in there!')
                    objecttype_name_list.append(name_val[0])
        print('totally', len(objecttype_name_list), 'object exist', '( total count of image', len(li), ')')
        

C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/bicycle_test.txt
totally 268 object exist ( total count of image 2686 )
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/bus_test.txt
totally 448 object exist ( total count of image 2686 )
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/car_test.txt
totally 992 object exist ( total count of image 2686 )
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/cat_test.txt
totally 1380 object exist ( total count of image 2686 )
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/cow_test.txt
totally 1577 object exist ( total count of image 2686 )
C:/Users/user/Desktop/programming_PROJECTS/dataset/pascal_voc_2006_new/test/ImageSets/dog_test.txt
totally 1947 object exist ( total count

In [25]:
objecttype_name_list

[]

In [13]:
#-------------------------------#
#   이미지 및 레이블 옮기기    #    
#-------------------------------#

for fname in objecttype_name_list :
    full_image_name = PNGImages_path_here + '/' + fname + '.png'
    full_annot_name = Annotations_path_here + '/' + fname + '.txt'
#    move_file_to_anotherfolder(full_image_name, Img_newpath_here, fname + '.png')
    move_file_to_anotherfolder(full_annot_name, Annotation_temp_here, fname + '.txt')

NameError: name 'objecttype_name_list' is not defined

In [216]:
from BboxObject import BboxObject
from BboxObject import FileData

#--------------------------------------------------------------------------#
#   필요없는 label data 를 제외한 데이터들을 pascal voc 형식으로 변환   #
#--------------------------------------------------------------------------#

for root, dirs, files in os.walk(Annotations_path_here):
    for fname in files: # files : annotation path 에 들어 있는 filename 의 list
        bboxobjects = [] # bbox 객체를 담는 변수. bbox 가 4개라면 객체 4개를 만들어 담을 것.
        
        full_annot_fname = root + '/' + fname
        full_image_name = PNGImages_path_here + '/' + fname + '.png'
        print('----------------start annotation file : ', fname ,'------------------')
        
        # just work : pascal annotation version 1.00
        # print(full_image_name, '\n', full_annot_name)
        # 단순한 parsing 작업
        myFile = open(full_annot_name, 'r')
        entire = myFile.read().split('\n\n')
        for textset in entire[4:] :
            textline_list = textset.split('\n')
            for textline in textline_list[2:] : # one textline per one object
                front_back = textline.split(':')
                if len(front_back) == 2:
                    print(front_back[0].split('\"')[1], end = '\t')
                    object_label = isStringExist(matching_dic, front_back[0])
                    object_loc, topleft, bottomright = bboxLocationParser(front_back[1])
                    print("::::", object_label, object_loc)
                    if object_label is None :
                        print(front_back[0], 'not used!')
                    else :
                        box = BboxObject(topleft, bottomright, front_back[0]) # topleft, bottomright
                        bboxobjects.append(box)
                else:
                    pass
            filedata = FileData(full_image_name, 0, 0, Img_newpath_here, 'clear')
            for bbox in bboxobjects :
                filedata.setObject(bbox)
            filedata.writeAndSave(full_image_name)
        myFile.close()

----------------start annotation file :  000021 ------------------
PASpersonTrunc	:::: person {'xmin': 309, 'ymin': 62, 'xmax': 341, 'ymax': 93}
PASpersonTruncDifficult	:::: person {'xmin': 382, 'ymin': 72, 'xmax': 403, 'ymax': 96}
PASpersonTruncDifficult	:::: person {'xmin': 138, 'ymin': 216, 'xmax': 159, 'ymax': 232}
PASpersonTrunc	:::: person {'xmin': 430, 'ymin': 213, 'xmax': 458, 'ymax': 259}
PASbus	:::: car {'xmin': 16, 'ymin': 17, 'xmax': 484, 'ymax': 360}


----------------start annotation file :  000100 ------------------
PASbus	:::: car {'xmin': 1, 'ymin': 106, 'xmax': 344, 'ymax': 341}
PAScar	:::: car {'xmin': 344, 'ymin': 214, 'xmax': 430, 'ymax': 276}
PAScar	:::: car {'xmin': 243, 'ymin': 242, 'xmax': 394, 'ymax': 330}
PAScarTruncDifficult	:::: car {'xmin': 336, 'ymin': 276, 'xmax': 407, 'ymax': 357}
PAScarTrunc	:::: car {'xmin': 43, 'ymin': 323, 'xmax': 289, 'ymax': 375}
PASpersonRearTruncDifficult	:::: person {'xmin': 415, 'ymin': 201, 'xmax': 435, 'ymax': 230}
PASperson



----------------start annotation file :  001228 ------------------
PASbusTrunc	:::: car {'xmin': 34, 'ymin': 65, 'xmax': 331, 'ymax': 472}
PAScarTrunc	:::: car {'xmin': 4, 'ymin': 370, 'xmax': 93, 'ymax': 497}
PAScarTruncDifficult	:::: car {'xmin': 5, 'ymin': 319, 'xmax': 40, 'ymax': 378}


----------------start annotation file :  001241 ------------------
PASbusRightTrunc	:::: car {'xmin': 21, 'ymin': 9, 'xmax': 420, 'ymax': 358}
PASpersonLeftTrunc	:::: person {'xmin': 59, 'ymin': 67, 'xmax': 167, 'ymax': 231}


----------------start annotation file :  001316 ------------------
PASbusTrunc	:::: car {'xmin': 78, 'ymin': 121, 'xmax': 245, 'ymax': 255}
PAScar	:::: car {'xmin': 320, 'ymin': 195, 'xmax': 390, 'ymax': 252}
PAScarTrunc	:::: car {'xmin': 411, 'ymin': 206, 'xmax': 436, 'ymax': 232}
PAScarTrunc	:::: car {'xmin': 433, 'ymin': 202, 'xmax': 467, 'ymax': 243}
PASpersonTrunc	:::: person {'xmin': 451, 'ymin': 180, 'xmax': 500, 'ymax': 374}
PASpersonDifficult	:::: person {'xmin': 51

----------------start annotation file :  002445 ------------------
PASmotorbikeFrontal	:::: None {'xmin': 12, 'ymin': 173, 'xmax': 174, 'ymax': 375}
PASpersonFrontal	:::: person {'xmin': 1, 'ymin': 40, 'xmax': 171, 'ymax': 358}
PASpersonFrontalTruncDifficult	:::: person {'xmin': 157, 'ymin': 131, 'xmax': 183, 'ymax': 199}
PASpersonFrontal	:::: person {'xmin': 185, 'ymin': 127, 'xmax': 212, 'ymax': 209}
PASpersonRear	:::: person {'xmin': 375, 'ymin': 149, 'xmax': 399, 'ymax': 215}
PASbus	:::: car {'xmin': 132, 'ymin': 54, 'xmax': 266, 'ymax': 205}


----------------start annotation file :  002468 ------------------
PASbusFrontalTrunc	:::: car {'xmin': 33, 'ymin': 2, 'xmax': 477, 'ymax': 322}


----------------start annotation file :  002479 ------------------
PASbus	:::: car {'xmin': 279, 'ymin': 150, 'xmax': 495, 'ymax': 293}
PAScarTrunc	:::: car {'xmin': 224, 'ymin': 213, 'xmax': 282, 'ymax': 249}


----------------start annotation file :  002513 ------------------
PASbus	:::: car {'x

----------------start annotation file :  004079 ------------------
PASbusTrunc	:::: car {'xmin': 68, 'ymin': 5, 'xmax': 361, 'ymax': 450}
PAScarRightTrunc	:::: car {'xmin': 1, 'ymin': 247, 'xmax': 68, 'ymax': 318}


----------------start annotation file :  004088 ------------------
PASbus	:::: car {'xmin': 152, 'ymin': 89, 'xmax': 395, 'ymax': 214}
PASmotorbikeRearDifficult	:::: None {'xmin': 432, 'ymin': 142, 'xmax': 449, 'ymax': 171}
PASperson	:::: person {'xmin': 275, 'ymin': 135, 'xmax': 299, 'ymax': 213}


----------------start annotation file :  004095 ------------------
PASbusTrunc	:::: car {'xmin': 66, 'ymin': 53, 'xmax': 500, 'ymax': 334}
PASbusTruncDifficult	:::: car {'xmin': 1, 'ymin': 125, 'xmax': 39, 'ymax': 234}


----------------start annotation file :  004097 ------------------
PASbus	:::: car {'xmin': 210, 'ymin': 65, 'xmax': 466, 'ymax': 302}


----------------start annotation file :  004123 ------------------
PASbusTrunc	:::: car {'xmin': 38, 'ymin': 5, 'xmax': 315, 

----------------start annotation file :  005005 ------------------
PASmotorbikeRearDifficult	:::: None {'xmin': 483, 'ymin': 146, 'xmax': 500, 'ymax': 177}
PASbus	:::: car {'xmin': 340, 'ymin': 110, 'xmax': 468, 'ymax': 169}
PASbus	:::: car {'xmin': 86, 'ymin': 38, 'xmax': 381, 'ymax': 273}
PASpersonFrontalTrunc	:::: person {'xmin': 1, 'ymin': 137, 'xmax': 33, 'ymax': 285}
PASpersonRear	:::: person {'xmin': 22, 'ymin': 135, 'xmax': 46, 'ymax': 188}
PAScarFrontal	:::: car {'xmin': 45, 'ymin': 138, 'xmax': 65, 'ymax': 159}
PAScarFrontalDifficult	:::: car {'xmin': 62, 'ymin': 136, 'xmax': 77, 'ymax': 154}
PAScarFrontalDifficult	:::: car {'xmin': 75, 'ymin': 136, 'xmax': 86, 'ymax': 149}
PAScarTruncDifficult	:::: car {'xmin': 80, 'ymin': 150, 'xmax': 91, 'ymax': 175}


----------------start annotation file :  005068 ------------------
PASbusTrunc	:::: car {'xmin': 34, 'ymin': 2, 'xmax': 375, 'ymax': 499}
PASperson	:::: person {'xmin': 126, 'ymin': 174, 'xmax': 215, 'ymax': 276}
PASperson	:

...

----------------start annotation file :  002207 ------------------
PASpersonFrontalTrunc	:::: person {'xmin': 323, 'ymin': 134, 'xmax': 455, 'ymax': 361}
PAShorseFrontal	:::: None {'xmin': 153, 'ymin': 99, 'xmax': 302, 'ymax': 375}


----------------start annotation file :  002208 ------------------
PASsheep	:::: None {'xmin': 281, 'ymin': 230, 'xmax': 378, 'ymax': 337}
PASperson	:::: person {'xmin': 268, 'ymin': 184, 'xmax': 346, 'ymax': 337}


----------------start annotation file :  002216 ------------------
PASbicycleRight	:::: None {'xmin': 51, 'ymin': 55, 'xmax': 184, 'ymax': 125}
PASpersonRight	:::: person {'xmin': 71, 'ymin': 6, 'xmax': 167, 'ymax': 120}


----------------start annotation file :  002228 ------------------
PASbicycleFrontal	:::: None {'xmin': 36, 'ymin': 108, 'xmax': 58, 'ymax': 161}
PASbicycleRear	:::: None {'xmin': 224, 'ymin': 173, 'xmax': 300, 'ymax': 296}
PASpersonFrontal	:::: person {'xmin': 33, 'ymin': 70, 'xmax': 60, 'ymax': 157}
PASpersonRear	:::: perso

----------------start annotation file :  003182 ------------------
PAShorseLeft	:::: None {'xmin': 245, 'ymin': 187, 'xmax': 370, 'ymax': 276}
PAShorseLeftTrunc	:::: None {'xmin': 315, 'ymin': 177, 'xmax': 422, 'ymax': 263}
PAShorseLeft	:::: None {'xmin': 127, 'ymin': 182, 'xmax': 253, 'ymax': 268}
PASpersonLeft	:::: person {'xmin': 167, 'ymin': 152, 'xmax': 197, 'ymax': 232}
PASpersonLeft	:::: person {'xmin': 291, 'ymin': 152, 'xmax': 323, 'ymax': 239}
PASpersonLeftDifficult	:::: person {'xmin': 338, 'ymin': 150, 'xmax': 373, 'ymax': 203}


----------------start annotation file :  003191 ------------------
PAScar	:::: car {'xmin': 170, 'ymin': 17, 'xmax': 344, 'ymax': 121}
PAScar	:::: car {'xmin': 367, 'ymin': 71, 'xmax': 518, 'ymax': 160}
PASperson	:::: person {'xmin': 227, 'ymin': 317, 'xmax': 261, 'ymax': 393}
PASperson	:::: person {'xmin': 160, 'ymin': 341, 'xmax': 191, 'ymax': 414}
PASperson	:::: person {'xmin': 150, 'ymin': 321, 'xmax': 177, 'ymax': 379}


----------------start 

----------------start annotation file :  003652 ------------------
PASbusRear	:::: car {'xmin': 41, 'ymin': 113, 'xmax': 228, 'ymax': 319}
PASbusTrunc	:::: car {'xmin': 321, 'ymin': 31, 'xmax': 500, 'ymax': 323}
PASpersonFrontal	:::: person {'xmin': 265, 'ymin': 210, 'xmax': 311, 'ymax': 312}


----------------start annotation file :  003668 ------------------
PASmotorbikeLeft	:::: None {'xmin': 210, 'ymin': 107, 'xmax': 268, 'ymax': 142}
PASpersonLeft	:::: person {'xmin': 224, 'ymin': 91, 'xmax': 254, 'ymax': 135}


----------------start annotation file :  003680 ------------------
PAShorseTrunc	:::: None {'xmin': 236, 'ymin': 262, 'xmax': 317, 'ymax': 472}
PAShorseTruncDifficult	:::: None {'xmin': 107, 'ymin': 181, 'xmax': 153, 'ymax': 273}
PAShorseTruncDifficult	:::: None {'xmin': 213, 'ymin': 164, 'xmax': 237, 'ymax': 205}
PASperson	:::: person {'xmin': 209, 'ymin': 140, 'xmax': 347, 'ymax': 382}
PASperson	:::: person {'xmin': 124, 'ymin': 127, 'xmax': 172, 'ymax': 240}
PASperson	:

PASpersonTrunc	:::: person {'xmin': 7, 'ymin': 16, 'xmax': 204, 'ymax': 333}


----------------start annotation file :  004488 ------------------
PASpersonRear	:::: person {'xmin': 339, 'ymin': 385, 'xmax': 406, 'ymax': 544}
PASpersonFrontal	:::: person {'xmin': 130, 'ymin': 354, 'xmax': 206, 'ymax': 526}


----------------start annotation file :  004497 ------------------
PAShorse	:::: None {'xmin': 146, 'ymin': 202, 'xmax': 373, 'ymax': 375}
PASperson	:::: person {'xmin': 110, 'ymin': 192, 'xmax': 196, 'ymax': 368}


----------------start annotation file :  004501 ------------------
PAShorse	:::: None {'xmin': 124, 'ymin': 183, 'xmax': 264, 'ymax': 437}
PASperson	:::: person {'xmin': 129, 'ymin': 90, 'xmax': 279, 'ymax': 330}


----------------start annotation file :  004549 ------------------
PAScarFrontalTrunc	:::: car {'xmin': 141, 'ymin': 101, 'xmax': 179, 'ymax': 145}
PAScarFrontal	:::: car {'xmin': 2, 'ymin': 101, 'xmax': 32, 'ymax': 122}
PAScarRearTrunc	:::: car {'xmin': 32, '

PAScarRear	:::: car {'xmin': 92, 'ymin': 42, 'xmax': 572, 'ymax': 450}
PAScarRearTrunc	:::: car {'xmin': 522, 'ymin': 23, 'xmax': 640, 'ymax': 131}
PASpersonRear	:::: person {'xmin': 484, 'ymin': 16, 'xmax': 514, 'ymax': 85}
PAScarTruncDifficult	:::: car {'xmin': 1, 'ymin': 107, 'xmax': 28, 'ymax': 305}


----------------start annotation file :  005168 ------------------
PASperson	:::: person {'xmin': 482, 'ymin': 222, 'xmax': 526, 'ymax': 328}
PASpersonDifficult	:::: person {'xmin': 348, 'ymin': 192, 'xmax': 375, 'ymax': 275}
PASperson	:::: person {'xmin': 355, 'ymin': 208, 'xmax': 391, 'ymax': 307}
PASbicycleRightDifficult	:::: None {'xmin': 49, 'ymin': 42, 'xmax': 95, 'ymax': 75}


----------------start annotation file :  005171 ------------------
PASbicycleLeft	:::: None {'xmin': 148, 'ymin': 158, 'xmax': 325, 'ymax': 275}
PASbicycleLeft	:::: None {'xmin': 61, 'ymin': 153, 'xmax': 170, 'ymax': 221}
PASpersonLeft	:::: person {'xmin': 175, 'ymin': 80, 'xmax': 288, 'ymax': 249}
PASper